In [6]:
import pandas as pd
import requests
from PIL import Image
from io import BytesIO
import numpy as np
import cv2
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tqdm import tqdm

In [4]:
df = pd.read_excel('chaussuretresbonetat.xlsx')
df2 = pd.read_excel('chaussurebonetat.xlsx')
df3 = pd.read_excel('chaussuresatisfaisant.xlsx')

In [5]:
df = df.iloc[:,4:6]
df2 = df2.iloc[:,4:6]
df3 = df3.iloc[:,4:6]

In [6]:
frames = [df,df2,df3]
result = pd.concat(frames,ignore_index=True)
result.to_csv('test.csv')

In [8]:
imageLst = []
labelLst = []

with requests.Session() as session:
    for i, url in enumerate(tqdm(result['image-src'])):
        try:
            resp = session.get(url)
            img = Image.open(BytesIO(resp.content))
            img_np = np.array(img)
            img_resized = cv2.resize(img_np, (150, 150))
            imageLst.append(img_resized)
            labelLst.append(result['Etat'][i])
        except:
            pass

# Convertir les listes en tableaux NumPy
x = np.array(imageLst)
y = np.array(labelLst)



  4%|▎         | 333/9287 [00:19<13:10, 11.32it/s]

In [ ]:
print(len(imageLst))
print(len(labelLst))

9287
9287


In [ ]:
np.save("array_X.npy", x)
np.save("array_Y.npy", y)

In [7]:
x = np.load("array_X.npy")
y = np.load("array_Y.npy")

In [9]:
# Encodage des étiquettes
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_one_hot = to_categorical(y_encoded)

In [25]:
print(label_encoder.classes_)

['bon etat' 'satisfaisant' 'très bon etat']


In [10]:
! pip install tensorflow-macos
! pip install tensorflow-metal

In [11]:
from tensorflow.python.client import device_lib

def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]

print(get_available_devices())


['/device:CPU:0', '/device:GPU:0']


2023-10-16 12:45:27.629040: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2 Pro
2023-10-16 12:45:27.629071: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 32.00 GB
2023-10-16 12:45:27.629082: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 10.67 GB
2023-10-16 12:45:27.629128: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-10-16 12:45:27.629149: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [12]:
from sklearn.model_selection import train_test_split

# Diviser les données en ensembles d'entraînement, de validation et de test
x_train, x_temp, y_train, y_temp = train_test_split(x, y_one_hot, test_size=0.3, random_state=42)
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.5, random_state=42)

from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models

# Charger VGG16 pré-entraîné sur ImageNet et sans les couches entièrement connectées
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# Geler les couches du modèle de base
for layer in base_model.layers:
    layer.trainable = False

# Créer le modèle personnalisé
model = models.Sequential()
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(3, activation='softmax'))  # Utiliser 'softmax' pour 3 classes

# Compilation du modèle
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Entraînement
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_val, y_val))

# Évaluer le modèle sur les données de test
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print("Test accuracy: {:.2f}%".format(test_accuracy * 100))


2023-10-16 12:45:31.404224: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-10-16 12:45:31.404243: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/10


2023-10-16 12:45:33.714285: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


204/204 [==============================] - 37s 160ms/step - loss: 54.2497 - accuracy: 0.3694 - val_loss: 38.9851 - val_accuracy: 0.3927
Epoch 2/10
204/204 [==============================] - 21s 104ms/step - loss: 28.6401 - accuracy: 0.4466 - val_loss: 38.9755 - val_accuracy: 0.3690
Epoch 3/10
204/204 [==============================] - 21s 104ms/step - loss: 22.3347 - accuracy: 0.4906 - val_loss: 29.3938 - val_accuracy: 0.4300
Epoch 4/10
204/204 [==============================] - 21s 104ms/step - loss: 18.7441 - accuracy: 0.5248 - val_loss: 22.6247 - val_accuracy: 0.4228
Epoch 5/10
204/204 [==============================] - 21s 104ms/step - loss: 17.7873 - accuracy: 0.5322 - val_loss: 29.8706 - val_accuracy: 0.4121
Epoch 6/10
204/204 [==============================] - 21s 104ms/step - loss: 16.4869 - accuracy: 0.5543 - val_loss: 14.7815 - val_accuracy: 0.5154
Epoch 7/10
204/204 [==============================] - 21s 104ms/step - loss: 13.9177 - accuracy: 0.5997 - val_loss: 15.2499 - val

In [13]:
# Sauvegarder le modèle entier
model.save("my_model.h5")


/Users/athena/Documents/exo/exo-env/lib/python3.11/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [34]:
# image = cv2.imread("chauss.jpeg")
# print(image)

In [17]:
!ls -lh chauss.jpeg

-rw-r--r--@ 1 athena  staff   5.9K Oct 16 12:50 chauss.jpeg


In [35]:
import cv2
import numpy as np

# Charger une image avec OpenCV
# image = cv2.imread("chauss.jpeg")
# image = cv2.imread("chauNeuv.jpeg")
# image = cv2.imread("chaBof.jpeg")
image = cv2.imread("IMG_6866.jpg")

# Redimensionner l'image à 150x150
image_resized = cv2.resize(image, (150, 150))

# Normaliser l'image
image_normalized = image_resized / 255.0

# Redimensionner l'image pour ajouter une dimension de batch
image_batch = np.expand_dims(image_normalized, axis=0)


In [36]:
# Effectuer une prédiction
predictions = model.predict(image_batch)

# Obtenir l'indice de la classe prédite
predicted_class = np.argmax(predictions[0])

print("Classe prédite:", predicted_class)


1/1 [==============================] - 0s 17ms/step
Classe prédite: 2
